In [1]:
import numpy as np
from myur import MyUR3e

robot = MyUR3e()
robot.move_gripper(255, 255, 255)
robot.clear_sim()
robot.trajectory_file = "cake_froster_trajectories.json"

In [23]:
# LOCATION VARIABLES
HOME = [0.3, 0.3, 0.3, 0, 0, 0]
CAKE_CENTER = [0.1, 0.3, 0.32]
CAKE_RADIUS = 0.026
CAKE_HEIGHT = 0.075
LOWEST_Z = 0.207 # true value #0.25 for testing

# FROSTING VARIABLES
FROSTING_HALFWAY_JOINTS = [4.13, -1.88, -1.56, -1.55, 0.67, -2.88]
FROSTING_START_JOINTS = [3.46, -2.02, -2.36, -1.89, -1.21, 3.16]
FROSTING_LENGTH = 0.15
FROSTING_DROP_HEIGHT = 0.006
NUM_SCOOP = 0
FROSTING_SEGMENT = 2

# Function to resolve out of bounds angles
def angle_clean(angle):
    angle = angle % 360
    if angle == 0:
        angle = 0.1
    return angle

# Motion to ice side of cake
def arc(divisor, segment=0, steps=100, center=CAKE_CENTER):
    # segment should be in range(divisor)
    # i.e. if divisor = 5, segment should be 0, 1, 2, 3, or 4

    angles = []
    for i in range(steps):
        ith_angle = 360 / divisor * (segment + i / steps)
        angles.append(ith_angle)
    arc_points = []
    for i, theta in enumerate(angles):
        SPATULA_ANGLE = 30 + 20*i/len(angles)

        
        # SPATULA CHARACTERISTICS
        SPATULA_RADIUS = 0.055 * np.cos(np.deg2rad(SPATULA_ANGLE))
        #SPATULA_OFFSET = 0.12
        EFFECTIVE_RADIUS = CAKE_RADIUS + SPATULA_RADIUS
        
        radius=EFFECTIVE_RADIUS
        x = center[0] + (radius) * np.cos(np.deg2rad(theta))
        y = center[1] + (radius) * np.sin(np.deg2rad(theta))
        if i == 0:
            x = center[0] + (radius) * np.cos(np.deg2rad(theta)) * 1.3
            y = center[1] + (radius) * np.sin(np.deg2rad(theta)) * 1.3
        elif i == (len(angles) - 1):
            x = center[0] + (radius) * np.cos(np.deg2rad(theta)) * 1.3
            y = center[1] + (radius) * np.sin(np.deg2rad(theta)) * 1.3

        arc_points.append(
            [x, y, LOWEST_Z, 0, 0.1, angle_clean(theta + 270 + SPATULA_ANGLE)]
        )

    start = list(arc_points[0])
    start[2] = start[2] + 0.1
    end = list(arc_points[-1])
    end[2] = end[2] + 0.1
    robot.move_global([start], time_step=0.6)
    robot.move_global(arc_points, time_step=(1, 0.04))
    robot.move_global([end], time_step=0.6)

# Motion to ice top of cake
def top_of_cake(joints_position, dx, dy):
    robot.move_joints([joints_position], time_step = 2)
    robot.move_global_r([[0.0,  dy, 0.0, 0.0, 0.0, 0.0],
                         [0.0, 0.0, 0.03, 0.0, 0.0, 0.0],
                         [-dx, -dy, -0.03, 0.0, 0.0, 0.0],
                         [0.0,  dy, 0.0, 0.0, 0.0, 0.0]], time_step = 0.8)

In [ ]:
ABOVE_CAKE_H = [-0.065, 0.301, 0.067, 50.071, 87.266, -100.51658272207979]
ABOVE_FROSTING_BEGINNING = [0.294, -0.102, 0.061, -29.745, 85.969, -87.3995992085437]

def main():
    # Move to frosting
    robot.move_joints('home_to_frosting_beginning', time_step = (2, 0.05), interp = 'spline')
    robot.move_global([ABOVE_FROSTING_BEGINNING], time_step = 1)
    robot.move_global_r([[0.0, 0.13, 0.0, 0.0, 0.0, 0.0]], time_step = 4)
    robot.move_joints('frosting_end_to_cake', time_step = (1, 0.05), interp = 'spline')

    SEGMENTS = 3
    for i in range(SEGMENTS):
        arc(SEGMENTS, segment = i)
        robot.move_joints('cake_to_frosting_beginning', time_step = (2, 0.05), interp = 'spline')
        robot.move_global([ABOVE_FROSTING_BEGINNING], time_step = 1)
        robot.move_global_r([[0.0, 0.0, -0.001 * (i + 1), 0.0, 0.0, 0.0]], time_step = 0.3)
        robot.move_global_r([[0.0, 0.13, 0.0, 0.0, 0.0, 0.0]], time_step = 4)
        robot.move_joints('frosting_end_to_cake', time_step = (1, 0.05), interp = 'spline')

    robot.move_joints('cake_v_to_cake_h', time_step = (1, 0.04), interp = 'spline')
    robot.move_global([ABOVE_CAKE_H], time_step = 1)
    robot.move_global_r([[0.0, 0.09, 0.0, 0.0, 0.0, 0.0]], time_step = 3)

In [ ]:
main()